## 3. Feature Engineering

Building on the exploratory plots, we quantify the observed temporal patterns by defining a compact set of **discriminative features** that enable a systematic characterisation of station usage types.

In [1]:
from data_io.loader.data_loader import DataLoader
from analysis.visualization.characterisation.features import build_feature_df

dl = DataLoader(city="Stadt_Heidelberg")
X = build_feature_df(dl)

### Double Peak Index (DPI):

Weekday hourly profiles of some stations exhibit a pronounced double-peak structure, typically associated with morning and evening commuting. The DPI quantifies this behaviour by identifying dominant morning (5–10 h) and evening (14–20 h) peaks and relating their magnitudes to the average midday level (8–14 h). Stations with clear and balanced commuting peaks yield high DPI values, whereas flat or single-peak profiles result in low scores.

Formally, let $p_m$ and $p_e$ denote the magnitudes of the morning and evening peaks at hours $h_m$ and $h_e$, and let $m$ be the average midday traffic level. The DPI is defined as

$$
\begin{align*}
\text{strength} &= \frac{(p_m - m) + (p_e - m)}{2} \\
\text{symmetry} &= 1 - \frac{|p_m - p_e|}{\max(p_m, p_e)} \\
\text{distance} &= \min\!\left(\frac{|h_e - h_m|}{10},\, 1\right) \\
\text{DPI} &= \max\!\left( \text{strength} \cdot \text{symmetry} \cdot \text{distance},\, 0 \right)
\end{align*}
$$

In [2]:
X_DPI = (
    X
    .select(["station", "DPI"])
    .sort("DPI", descending=True)
)

X_DPI

station,DPI
str,f64
"""Ernst-Walz-Brücke Querschnitt""",0.033678
"""Mannheimer Straße""",0.033604
"""Bahnstadtpromenade""",0.033136
"""Hardtstraße""",0.032738
"""Ziegelhäuser Landstraße""",0.030285
…,…
"""Eppelheimer Str. Querschnitt""",0.012655
"""Thedor-Heuss-Brücke Querschnit…",0.011926
"""Berliner Straße Querschnitt""",0.009011


### Weekend Shape Difference (WSD):

Differences between weekday and weekend hourly traffic patterns provide an additional discriminator between usage types. To capture this effect, we compare the *shape* of the weekday and weekend hourly profiles.

Let $\mathbf{p}^{wd}$ and $\mathbf{p}^{we}$ denote the weekday and weekend hourly profiles, normalised to sum to one. The Weekend Shape Difference is defined as

$$
\text{WSD} =
\left\lVert
\frac{\mathbf{p}^{wd}}{\sum_h p^{wd}_h}
-
\frac{\mathbf{p}^{we}}{\sum_h p^{we}_h}
\right\rVert_2
$$

High values indicate strong differences between weekday and weekend usage patterns, while low values suggest similar diurnal structures.

In [3]:
X_WSD = (
    X
    .select(["station", "WSD"])
    .sort("WSD", descending=True)
)

X_WSD

station,WSD
str,f64
"""Schlierbacher Landstraße""",0.085375
"""Ziegelhäuser Landstraße""",0.080479
"""Bahnstadtpromenade""",0.078063
"""Hardtstraße""",0.076721
"""Mannheimer Straße""",0.072783
…,…
"""Berliner Straße Querschnitt""",0.057259
"""Eppelheimer Str. Querschnitt""",0.05384
"""Thedor-Heuss-Brücke Querschnit…",0.050217


### Seasonal Drop Index (SDI):

Seasonality provides a further discriminator between recreational and utilitarian stations. The SDI quantifies the relative drop between high-usage and low-usage months based on the monthly index.

Let $I_m$ denote the monthly index values of a station. Using upper and lower quantiles to ensure robustness, the Seasonal Drop Index is defined as

$$
\begin{align*}
q_{90} &= \text{quantile}_{0.9}(I_m) \\
q_{10} &= \text{quantile}_{0.1}(I_m) \\
\text{SDI} &= \frac{q_{90} - q_{10}}{q_{90}}
\end{align*}
$$

High SDI values indicate strong seasonal variation, while low values correspond to relatively stable, year-round usage.


In [4]:
X_SDI = (
    X
    .select(["station", "SDI"])
    .sort("SDI", descending=True)
)

X_SDI

station,SDI
str,f64
"""Schlierbacher Landstraße""",0.636018
"""Ziegelhäuser Landstraße""",0.531836
"""Mannheimer Straße""",0.504696
"""Bahnstadtpromenade""",0.487909
"""Kurfürstenanlage Querschnitt""",0.482909
…,…
"""Liebermannstraße""",0.375826
"""Ernst-Walz-Brücke Querschnitt""",0.34561
"""Thedor-Heuss-Brücke Querschnit…",0.33746
